# Tutorial sobre os principais encoders e suas formas de uso.

Este notebook contém o resumo do artigo que escrevi no link:

No mundo real, muitos dos nossos dados podem ser categóricos, temporais, textuais ou mesmo complexos, como imagens ou áudio. Os codificadores de características ajudam a converter esses tipos complexos de dados em formatos mais simples, como números, que podem ser analisados por modelos de machine learning.

## Dataset usado nos exemplos

Vamos criar um dataset simples com variáveis categóricas e um target. Nos exeplos não usaremos a divisão de treino e teste, mas lembre-se, *É SEMPRE IMPORTANTE SE ATENTAR À DIVISÃO DE TREINO E TESTE* (falaremos mais sobre em breve.)

In [95]:
import pandas as pd

df = pd.DataFrame({'animal': ['cachorro', 'elefante', 'galinha', 'leao', 'ser humano', 'cachorro', 'gato', 'ser humano', 'elefante', 'pardal'],
                   'locomocao' : ['quadrupede', 'quadrupede','bipede',  'quadrupede', 'bipede',  'quadrupede',  'quadrupede', 'bipede', 'quadrupede', 'bipede'],
                   'target': [0, 0, 1, 0, 0, 0, 0, 0, 0, 1]})

## O que são encoders?

A codificação de recursos (feature encoding) é uma etapa essencial no processo de aprendizado de máquina que envolve a conversão de dados brutos em um formato que pode ser entendido por algoritmos de aprendizado de máquina. Essencialmente, é um processo de transformação dos dados de entrada em um formato que é mais adequado para o modelo específico que você está treinando.

Muitos algoritmos de aprendizado de máquina, particularmente os que envolvem aprendizado supervisionado, exigem que os dados de entrada sejam numéricos. No entanto, muitos conjuntos de dados contêm recursos categóricos que são expressos como texto, em vez de números. Esses recursos categóricos precisam ser codificados como números antes que possam ser usados em um modelo de aprendizado de máquina.

Existem várias técnicas comuns para codificar recursos categóricos, cada uma com seus próprios prós e contras. Neste tutorial, vou mostrar a aplicação dos principais e seus prós e contras.

## Como e onde usá-los?

A codificação de recursos é uma parte essencial do pré-processamento de dados para aprendizado de máquina e escolher a técnica de codificação correta pode ter um impacto significativo no desempenho do seu modelo.

É crucial ajustar (fit) os codificadores aos dados de treinamento e depois aplicá-los (transform) tanto aos dados de treinamento quanto aos de teste. Isso ocorre porque se os codificadores forem ajustados aos dados de teste, isso permitirá o vazamento de informações dos dados de teste para o modelo de aprendizado de máquina, que pode resultar em uma avaliação excessivamente otimista do desempenho do modelo. Abaixo é demonstrado o jeito correto de aplicar um dos encoders que discutiremos a seguir. Vale lembrar que este que está sendo usado não necessariamente é o melhor, esotu usando como exemplo por ser um dos mais conhecidos.

In [7]:
from category_encoders import OneHotEncoder
from sklearn.model_selection import train_test_split

# Primeiro, dividimos os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Em seguida, ajustamos o codificador aos dados de treinamento
encoder = OneHotEncoder(cols=['animal', 'locomocao'])
encoder.fit(X_train)

# Finalmente, transformamos os dados de treinamento e teste
X_train_encoded = encoder.transform(X_train)
X_test_encoded = encoder.transform(X_test)

In [8]:
X_train.head()

,animal,locomocao
5,ser humano,bipede
2,elefante,quadrupede
4,leao,quadrupede
3,passaro,bipede


In [9]:
X_train_encoded.head()

,animal_1,animal_2,animal_3,animal_4,locomocao_1,locomocao_2
5,1,0,0,0,1,0
2,0,1,0,0,0,1
4,0,0,1,0,0,1
3,0,0,0,1,1,0


Observe que o codificador é ajustado (fit) apenas aos dados de treinamento, mas então é aplicado (transform) aos dados de treinamento e teste. Isso garante que o mesmo esquema de codificação seja usado para ambos os conjuntos de dados, mas que nenhuma informação dos dados de teste seja usada para criar esse esquema de codificação.

Outra coisa muito importante, quando você vai aplicar seu modelo a novos dados no futuro (por exemplo, em um sistema de produção), você precisará aplicar o mesmo esquema de codificação a esses novos dados. Isso significa que você precisará manter o codificador que você ajustou aos seus dados de treinamento, para que possa ser reutilizado para transformar quaisquer novos dados que o seu modelo irá prever.

## Codificação One-Hot

A codificação one-hot é uma das técnicas de codificação mais comuns para lidar com variáveis categóricas. Ela transforma cada categoria em uma nova coluna e atribui um valor binário de 1 ou 0. Cada inteiro é representado como um vetor binário. Todas as posições binárias nesse vetor são zero, exceto a posição do índice do inteiro, que é marcada como um. Esse é o motivo pelo qual é conhecida como codificação "one-hot".

Este é um método de codificação que cria novas colunas indicando a presença (ou ausência) de cada valor possível em dados categóricos originais. Por exemplo, na coluna coluna "animal" que pode ter os valores "cachorro", "gato", "pássaro", etc.. a codificação one-hot criará três novas colunas: "animal_cachorro", "animal_gato" e "animal_passaro". Se a linha original tiver "cachorro" como valor, a coluna "animal_cachorro" será 1 e todas as outras serão 0. 

- Prós: Simples de entender e implementar. Funciona bem com variáveis categóricas de baixa cardinalidade (número de elementos do conjunto, quantidade de animais diferentes na coluna).

- Contras: Cria muitas novas colunas se a variável categórica tiver muitos valores únicos (alta cardinalidade), o que pode levar à "maldição da dimensionalidade", ou seja, para um dado tamanho de amostras, existe um número máximo de características a partir do qual o desempenho do classificador irá degradar, ao invés de melhorar. Para saber mais sobre isso, sugiro estudar sobre o trade-off em modelos de ML. Aqui está um artigo escrito por mim sobre o assunto: https://medium.com/@pedrorp/balancing-trade-offs-in-machine-learning-algorithms-a-contextual-approach-7a4c382846a3.

Outro ponto a se destacar, é que a  multicolinearidade pode ser um problema ao usar a codificação one-hot, já que as variáveis codificadas estão altamente correlacionadas. Isso pode ser evitado ao remover uma das colunas codificadas (também conhecida como "dummy variable trap").

### Exemplo de código

In [136]:
from category_encoders import OneHotEncoder
from sklearn.model_selection import train_test_split

df = pd.DataFrame({
    'animal': ['cachorro', 'elefante', 'galinha', 'leao', 'ser humano', 'cachorro', 'gato', 'ser humano', 'elefante', 'pardal', 'macaco', 'cachorro'],
    'locomocao' : ['quadrupede', 'quadrupede','bipede',  'quadrupede', 'bipede',  'quadrupede',  'quadrupede', 'bipede', 'quadrupede', 'bipede', 'bipede', 'quadrupede'],
    'target': [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
})

X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

encoder = OneHotEncoder(cols=['animal', 'locomocao'])

# Treino
encoder.fit(X_train)
X_train_encoded = encoder.transform(X_train)

# Teste
X_test_encoded = encoder.transform(X_test)

In [137]:
X_train_encoded

,animal_1,animal_2,animal_3,animal_4,animal_5,animal_6,locomocao_1,locomocao_2
10,1,0,0,0,0,0,1,0
2,0,1,0,0,0,0,1,0
1,0,0,1,0,0,0,0,1
8,0,0,1,0,0,0,0,1
4,0,0,0,1,0,0,1,0
7,0,0,0,1,0,0,1,0
3,0,0,0,0,1,0,0,1
6,0,0,0,0,0,1,0,1


In [138]:
X_test_encoded

,animal_1,animal_2,animal_3,animal_4,animal_5,animal_6,locomocao_1,locomocao_2
5,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,1,0


## Codificação Label/Ordinal

 Este método de codificação converte cada valor único em um número diferente. É útil quando há uma ordem intrínseca nos valores da categoria. Por exemplo, para uma coluna "tamanho" com valores "pequeno", "medio" e "grande", podemos codificar "pequeno" como 1, "medio" como 2 e "grande" como 3.
 
- Prós: Simples de implementar. Não aumenta a dimensionalidade dos dados.

- Contras: Pode introduzir uma relação de ordem que não existe nos dados originais, o que pode confundir o modelo.

### Exemplo de código

In [141]:
from category_encoders import OrdinalEncoder
from sklearn.model_selection import train_test_split

df = pd.DataFrame({
    'animal': ['cachorro', 'elefante', 'galinha', 'leao', 'ser humano', 'cachorro', 'gato', 'ser humano', 'elefante', 'pardal', 'macaco', 'cachorro'],
    'locomocao' : ['quadrupede', 'quadrupede','bipede',  'quadrupede', 'bipede',  'quadrupede',  'quadrupede', 'bipede', 'quadrupede', 'bipede', 'bipede', 'quadrupede'],
    'target': [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
})

X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

encoder = OrdinalEncoder(cols=['animal', 'locomocao'])

# Treino
encoder.fit(X_train)
X_train_encoded = encoder.transform(X_train)

# Teste
X_test_encoded = encoder.transform(X_test)


In [143]:
X_train_encoded

,animal,locomocao
10,1,1
2,2,1
1,3,2
8,3,2
4,4,1
7,4,1
3,5,2
6,6,2


In [144]:
X_test_encoded

,animal,locomocao
5,-1.0,2
0,-1.0,2
9,-1.0,1


Note que um valor de -1 após a codificação geralmente indica que o codificador encontrou uma categoria no conjunto de teste que não estava presente no conjunto de treinamento. Este é um problema comum ao trabalhar com dados categóricos, pois nem sempre todas as categorias possíveis estão presentes em todos os subconjuntos de dados.

## Codificação Binária

Este método primeiro atribui números inteiros a cada categoria, como na codificação de rótulo (Label). Em seguida, ele converte esses inteiros em binário. Por exemplo, se tivermos as categorias "Cachorro", "Gato" e "Pássaro", podemos atribuir-lhes os inteiros 1, 2 e 3, respectivamente. Em binário, estes são 01, 10 e 11. Portanto, precisaríamos de duas novas colunas para representar estes dados.

- Prós: Útil para lidar com variáveis categóricas de alta cardinalidade sem aumentar muito a dimensionalidade (ainda sim é notório o aumento da dimensionalidade em relação a outros encoders).

- Contras: Mais complexo de entender e implementar do que one-hot ou label encoding.

### Exemplo de código

In [158]:
from category_encoders import BinaryEncoder
from sklearn.model_selection import train_test_split

df = pd.DataFrame({
    'animal': ['cachorro', 'elefante', 'galinha', 'leao', 'ser humano', 'cachorro', 'gato', 'ser humano', 'elefante', 'pardal', 'macaco', 'cachorro'],
    'locomocao' : ['quadrupede', 'quadrupede','bipede',  'quadrupede', 'bipede',  'quadrupede',  'quadrupede', 'bipede', 'quadrupede', 'bipede', 'bipede', 'quadrupede'],
    'target': [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
})

X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

encoder = BinaryEncoder(cols=['animal', 'locomocao'])

# Treino
encoder.fit(X_train)
X_train_encoded = encoder.transform(X_train)

# Teste
X_test_encoded = encoder.transform(X_test)

In [153]:
X_train_encoded

,animal_0,animal_1,animal_2,locomocao_0,locomocao_1
8,0,0,1,0,1
5,0,1,0,0,1
2,0,1,1,1,0
1,0,0,1,0,1
11,0,1,0,1,1
4,1,0,0,1,0
7,1,0,0,1,0
3,1,0,1,0,1
6,1,1,0,0,1


In [154]:
X_test_encoded

,animal_0,animal_1,animal_2,locomocao_0,locomocao_1
10,0,0,0,1,0
9,0,0,0,1,0
0,0,1,0,0,1


Importante ressaltar, a codificação binária não gera uma representação binária na forma que você pode estar esperando. Em vez de codificar categorias diretamente em seu equivalente binário (por exemplo, 1, 10, 11, 100), a codificação binária segue um processo um pouco diferente para minimizar o número de novas colunas que são adicionadas ao conjunto de dados.

Em codificação binária normal, poderíamos representar essas categorias como 1, 10 e 11, etc. No entanto, a codificação binária usada em machine learning resolve esse problema ao usar um número mínimo de bits que pode representar todas as categorias. Assim, em vez de codificar as categorias diretamente como binários, a codificação em um exemplo aleatório seria algo como:

- azul: 001
- vermelho: 010
- verde: 011
- roxo: 100

## Codificação de Helmert

A codificação de Helmert, também conhecida como codificação contrastante, é uma forma de transformar variáveis categóricas em uma forma que possa ser usada por algoritmos de aprendizado de máquina. Ao contrário de outras formas de codificação, como a codificação one-hot, a codificação de Helmert compara cada nível de uma variável categórica aos níveis subsequentes. Ela é chamada de "contraste" porque a codificação destaca a diferença entre cada nível da variável categórica e a média dos níveis subsequentes. Isso é útil para modelos de regressão e outros modelos que podem usar essas diferenças.

A codificação de Helmert é realizada em várias etapas:

- As categorias são ordenadas de alguma maneira. Isso poderia ser a ordem na qual as categorias aparecem no conjunto de dados, a ordem alfabética ou alguma outra ordem.

- Para a primeira categoria, uma codificação de -1 é aplicada para todas as observações dessa categoria e 0 para todas as outras.

- Para a segunda categoria, uma codificação de 1 é aplicada para todas as observações dessa categoria, -1 para a primeira e 0 para todas as outras.

- Para a terceira categoria, se houver, uma codificação de 2 é aplicada para todas as observações dessa categoria, 0 para a primeira e segunda e -2 para todas as outras.

E assim por diante para as outras categorias.

Note que a codificação de Helmert resulta em valores negativos, ao contrário da codificação one-hot, por exemplo, que gera apenas valores 0 e 1. Um aspecto importante a se considerar ao usar a codificação de Helmert é que ela assume que existe uma ordem nas categorias. Portanto, ela é mais adequada para variáveis categóricas ordinais do que para variáveis categóricas nominais.

Além disso, vale ressaltar que as variáveis categóricas codificadas com Helmert podem ser mais difíceis de interpretar do que as variáveis categóricas codificadas usando métodos mais simples, como a codificação one-hot.

- Prós: Fornece comparações entre cada categoria e a média das categorias subsequentes. Útil para análises estatísticas avançadas.

- Contras: Difícil de entender e implementar. Não é adequado para todas as situações, pois pode introduzir complexidade desnecessária.

### Exemplo de código

In [155]:
from category_encoders import HelmertEncoder
from sklearn.model_selection import train_test_split

df = pd.DataFrame({
    'animal': ['cachorro', 'elefante', 'galinha', 'leao', 'ser humano', 'cachorro', 'gato', 'ser humano', 'elefante', 'pardal', 'macaco', 'cachorro'],
    'locomocao' : ['quadrupede', 'quadrupede','bipede',  'quadrupede', 'bipede',  'quadrupede',  'quadrupede', 'bipede', 'quadrupede', 'bipede', 'bipede', 'quadrupede'],
    'target': [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
})

X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

encoder = HelmertEncoder(cols=['animal', 'locomocao'])

# Treino
encoder.fit(X_train)
X_train_encoded = encoder.transform(X_train)

# Teste
X_test_encoded = encoder.transform(X_test)

c:\Users\Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
c:\Users\Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",
c:\Users\Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\category_encoders\base_contrast_encoder.py:126: FutureWarning: Intercept column might not be added anymore in future releases (c.f. issue #370)
  warnings.warn("Intercept column might not be added anymore in future releases (c.f. issue #370)",


In [156]:
X_train_encoded

,intercept,animal_0,animal_1,animal_2,animal_3,animal_4,locomocao_0
8,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
5,1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,1,0.0,2.0,-1.0,-1.0,-1.0,1.0
1,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
11,1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,1,0.0,0.0,3.0,-1.0,-1.0,1.0
7,1,0.0,0.0,3.0,-1.0,-1.0,1.0
3,1,0.0,0.0,0.0,4.0,-1.0,-1.0
6,1,0.0,0.0,0.0,0.0,5.0,-1.0


In [157]:
X_test_encoded

,intercept,animal_0,animal_1,animal_2,animal_3,animal_4,locomocao_0
10,1,0.0,0.0,0.0,0.0,0.0,1.0
9,1,0.0,0.0,0.0,0.0,0.0,1.0
0,1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0



A coluna "intercept" criada pelo HelmertEncoder é uma coluna de uns (1) que é adicionada ao conjunto de dados codificado. Isso é feito para incluir um termo de intercepto nos modelos de regressão que você pode ajustar aos seus dados posteriormente. Em um modelo de regressão, o intercepto (também conhecido como constante ou bias) é o valor esperado da variável dependente quando todas as variáveis independentes são iguais a zero. Isso pode não ter sentido para todas as variáveis independentes, especialmente se elas não puderem assumir um valor de zero. No caso da codificação Helmert, o termo de intercepto representa a média global para a variável categórica que está sendo codificada. A inclusão desse termo de intercepto permite que o modelo faça comparações entre cada nível da variável categórica e a média global.

Se você não deseja essa coluna em seu conjunto de dados, você pode remover essa coluna após a codificação, ou configurar drop_invariant=True ao instanciar o codificador, o que fará com que o codificador remova colunas invariantes (colunas que têm o mesmo valor em todas as linhas) durante o ajuste e a transformação:

## Codificação de Hashing

A codificação de Hashing é uma técnica de codificação eficiente para variáveis categóricas. Ela usa a função de hashing para representar categorias usando menos dimensões que podem ser necessárias em outros métodos de codificação, como a codificação One-Hot. A codificação de Hashing funciona da seguinte forma:

- Uma função de hash é escolhida. Uma função de hash é uma função que mapeia dados de tamanho arbitrário para dados de tamanho fixo. Em outras palavras, não importa o quão grande ou pequeno seja o input, o output sempre terá o mesmo tamanho.

- Cada categoria é passada pela função de hash, que retorna um índice inteiro. Este índice é então usado para representar a categoria.

- A dimensão do espaço de características resultante (ou seja, o número de colunas resultantes) é pré-definida, portanto, a função de hash deve retornar índices que estejam dentro desse espaço de características. Se houver mais categorias do que o espaço de características permitir, diferentes categorias podem ser mapeadas para o mesmo índice. Chamamos isso de "colisão".

Prós: Útil para lidar com variáveis categóricas de alta cardinalidade. O número de colunas adicionais é controlável pelo usuário.

Contras: Como múltiplas categorias podem ser mapeadas para o mesmo hash, pode ocorrer colisões, o que pode diminuir a precisão do modelo.

### Exemplo de código

In [160]:
from category_encoders import HashingEncoder
from sklearn.model_selection import train_test_split

df = pd.DataFrame({
    'animal': ['cachorro', 'elefante', 'galinha', 'leao', 'ser humano', 'cachorro', 'gato', 'ser humano', 'elefante', 'pardal', 'macaco', 'cachorro'],
    'locomocao' : ['quadrupede', 'quadrupede','bipede',  'quadrupede', 'bipede',  'quadrupede',  'quadrupede', 'bipede', 'quadrupede', 'bipede', 'bipede', 'quadrupede'],
    'target': [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
})

X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

encoder = HashingEncoder(cols=['animal', 'locomocao'], n_components=8)

# Treino
encoder.fit(X_train, y_train)
X_train_encoded = encoder.transform(X_train)

# Teste
X_test_encoded = encoder.transform(X_test)

In [161]:
X_train_encoded

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7
8,1,0,0,0,1,0,0,0
5,0,0,1,0,1,0,0,0
2,1,0,0,0,0,1,0,0
1,1,0,0,0,1,0,0,0
11,0,0,1,0,1,0,0,0
4,2,0,0,0,0,0,0,0
7,2,0,0,0,0,0,0,0
3,1,0,0,0,1,0,0,0
6,0,0,0,1,1,0,0,0


In [162]:
X_test_encoded

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7
10,1,0,1,0,0,0,0,0
9,1,0,0,1,0,0,0,0
0,0,0,1,0,1,0,0,0


Note que o HashingEncoder tem um parâmetro n_components que determina o número de recursos que serão criados pela codificação. Você pode ajustar esse parâmetro de acordo com o número de características que espera que sua variável categórica tenha. No entanto, se o número de componentes for muito pequeno, você pode ter colisões de hash, onde diferentes categorias recebem o mesmo código hash. Se o número de componentes for muito grande, você pode ter muitas características e potencialmente overfit seu modelo.

## Codificação de frequência

A codificação de frequência é uma maneira de transformar variáveis categóricas substituindo as categorias pelo número de vezes que aparecem no conjunto de dados. Isso pode ser útil quando a frequência de uma categoria é mais informativa que a categoria em si. Para cada categoria na característica, a frequência (ou proporção) da categoria no conjunto de dados é calculada e então substitui a instância da categoria.

- Prós: Uma vantagem da codificação de frequência é que ela pode capturar informações sobre a distribuição das categorias no conjunto de dados. Isso pode ser informativo para o modelo se a frequência da categoria tiver uma relação com o resultado.

- Uma desvantagem é que a codificação de frequência pode ser sensível à distribuição das categorias no conjunto de dados. Se a distribuição das categorias nos dados de treinamento for significativamente diferente da distribuição nos dados de teste, o modelo pode ter um desempenho pior nos dados de teste.

Além disso, a codificação de frequência pode introduzir uma relação de ordem entre as categorias baseada na frequência. Em alguns casos, isso pode ser desejável, mas em outros casos, pode introduzir ruído no modelo.

In [163]:
from sklearn.model_selection import train_test_split

df = pd.DataFrame({
    'animal': ['cachorro', 'elefante', 'galinha', 'leao', 'ser humano', 'cachorro', 'gato', 'ser humano', 'elefante', 'pardal', 'macaco', 'cachorro'],
    'locomocao' : ['quadrupede', 'quadrupede','bipede',  'quadrupede', 'bipede',  'quadrupede',  'quadrupede', 'bipede', 'quadrupede', 'bipede', 'bipede', 'quadrupede'],
    'target': [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
})

X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

animal_freq = X_train['animal'].value_counts() / len(X_train)
locomocao_freq = X_train['locomocao'].value_counts() / len(X_train)

X_train_encoded = X_train.copy()
X_train_encoded['animal'] = X_train['animal'].map(animal_freq)
X_train_encoded['locomocao'] = X_train['locomocao'].map(locomocao_freq)

X_test_encoded = X_test.copy()
X_test_encoded['animal'] = X_test['animal'].map(animal_freq)
X_test_encoded['locomocao'] = X_test['locomocao'].map(locomocao_freq)

# Preenche NaNs para categorias não vistas durante o treinamento com uma frequência de 0
X_train_encoded.fillna(0, inplace=True)
X_test_encoded.fillna(0, inplace=True)

In [164]:
X_train_encoded

,animal,locomocao
8,0.222222,0.666667
5,0.222222,0.666667
2,0.111111,0.333333
1,0.222222,0.666667
11,0.222222,0.666667
4,0.222222,0.333333
7,0.222222,0.333333
3,0.111111,0.666667
6,0.111111,0.666667


In [165]:
X_test_encoded

,animal,locomocao
10,0.000000,0.333333
9,0.000000,0.333333
0,0.222222,0.666667


Se uma categoria do conjunto de teste não foi vista durante o treinamento, ela é substituída por 0, já que sua frequência no conjunto de treinamento é 0.

## Codificação alvo (Target Encoder)

A codificação Target (também conhecida como codificação de média ou codificação de resposta) é uma maneira de transformar variáveis categóricas usando informações do target (ou resposta). Para cada categoria, calculamos a média do valor alvo. Então, substituímos cada instância da categoria por essa média.

A vantagem da codificação Target é que ela pode capturar informações sobre a relação entre a categoria e o target. No entanto, ela tem um grande problema: pode causar vazamento de dados (ou seja, informação do conjunto de teste vazando para o conjunto de treinamento ou vice-versa). Por exemplo, se calculamos a média do target para uma categoria usando o conjunto inteiro de dados, então essa média terá informação tanto do conjunto de treinamento quanto do conjunto de teste. Para evitar isso, é comum calcular a média do target apenas no conjunto de treinamento e usar essa média para substituir as categorias tanto no conjunto de treinamento quanto no conjunto de teste.

Além disso, a codificação Target pode causar sobreajuste se algumas categorias tiverem muito poucas ocorrências. Uma maneira de mitigar isso é adicionar algum tipo de "suavização". Por exemplo, em vez de calcular a média do target diretamente, poderíamos calcular uma média ponderada que também leva em consideração a média global do target.

- Prós: Pode capturar informação relacionada à variável alvo que os outros métodos de codificação não conseguem.

- Contras: Pode causar vazamento de dados, levando a resultados superestimados se não for feito corretamente. Também pode levar a overfitting.

In [166]:
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split

df = pd.DataFrame({
    'animal': ['cachorro', 'elefante', 'galinha', 'leao', 'ser humano', 'cachorro', 'gato', 'ser humano', 'elefante', 'pardal', 'macaco', 'cachorro'],
    'locomocao' : ['quadrupede', 'quadrupede','bipede',  'quadrupede', 'bipede',  'quadrupede',  'quadrupede', 'bipede', 'quadrupede', 'bipede', 'bipede', 'quadrupede'],
    'target': [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
})

X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

encoder = TargetEncoder(cols=['animal', 'locomocao'])

# Treino
encoder.fit(X_train, y_train)
X_train_encoded = encoder.transform(X_train)

# Teste
X_test_encoded = encoder.transform(X_test)

In [167]:
X_train_encoded

,animal,locomocao
8,0.095350,0.089132
5,0.095350,0.089132
2,0.226763,0.145437
1,0.095350,0.089132
11,0.095350,0.089132
4,0.095350,0.145437
7,0.095350,0.145437
3,0.096655,0.089132
6,0.096655,0.089132


In [168]:
X_test_encoded

,animal,locomocao
10,0.111111,0.145437
9,0.111111,0.145437
0,0.095350,0.089132


Este código irá adicionar uma nova coluna 'animal_encoded' aos conjuntos de dados de treinamento e teste, onde cada animal é substituído pela média do target para aquele animal. Essas médias são calculadas apenas com base nos dados de treinamento. As colunas 'animal' originais ainda estarão disponíveis nos DataFrames. Se uma categoria do conjunto de teste não foi vista durante o treinamento, ela é substituída pela média geral de 'target' no conjunto de treinamento.

Lembre-se, é muito importante que você não ajuste o codificador nos dados de teste - apenas use o método transform. Ajustar o codificador nos dados de teste pode levar a resultados que não são representativos da performance do modelo em dados não vistos, já que isso estaria usando informação que não estaria disponível em uma situação real de previsão.

## Codificação de Retorno de Probabilidade (Weight of Evidence Encoding)

A Codificação de Retorno de Probabilidade, também conhecida como Weight of Evidence (WoE) Encoding, é uma técnica utilizada principalmente em estatísticas e análise de crédito, mas também é útil em aprendizado de máquina para transformar variáveis categóricas.

A Codificação de Retorno de Probabilidade transforma uma variável categórica em um novo valor que captura a informação sobre o resultado do target associado a cada categoria. Para um target binário (como 0/1 ou Sim/Não), WoE é calculado como o log da divisão da probabilidade de sucesso pela probabilidade de falha.

WoE = ln( (n de eventos positivos / n total de eventos positivos) / (n de eventos negativos / n total de eventos negativos) )

Aqui, "eventos positivos" se refere ao número de vezes que o target é a classe de interesse (por exemplo, 1 em um problema de classificação binária), e "eventos negativos" se refere ao número de vezes que o target é a outra classe. Os valores de WoE são então usados para substituir as categorias na variável original. Uma vantagem desta abordagem é que ela cria uma única coluna de números reais que mantém a informação do target associada a cada categoria. Isso pode ser útil para algoritmos de aprendizado de máquina que funcionam melhor com entradas numéricas.

No entanto, como outras técnicas de codificação que usam informações do target, a Codificação de Retorno de Probabilidade pode levar ao vazamento de dados se não for feita corretamente. Para evitar isso, é importante calcular os valores de WoE usando apenas o conjunto de treinamento e aplicar esses valores ao conjunto de teste.

- Prós: Captura a relação entre a categoria e a variável alvo, que pode ser especialmente útil para variáveis binárias.

- Contras: Pode causar vazamento de dados se não for feito corretamente. Além disso, sua aplicação pode ser bastante limitada para casos de uso específicos (como credit scoring).

In [172]:
from category_encoders import WOEEncoder
from sklearn.model_selection import train_test_split

df = pd.DataFrame({
    'animal': ['cachorro', 'elefante', 'galinha', 'leao', 'ser humano', 'cachorro', 'gato', 'ser humano', 'elefante', 'pardal', 'macaco', 'cachorro'],
    'locomocao' : ['quadrupede', 'quadrupede','bipede',  'quadrupede', 'bipede',  'quadrupede',  'quadrupede', 'bipede', 'quadrupede', 'bipede', 'bipede', 'quadrupede'],
    'target': [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]
})

X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

encoder = WOEEncoder()

# Treino
encoder.fit(X_train['animal'], y_train)
X_train['animal_encoded'] = encoder.transform(X_train['animal'])

# Teste
X_test['animal_encoded'] = encoder.transform(X_test['animal'])

In [173]:
X_train_encoded

,animal,locomocao
8,0.095350,0.089132
5,0.095350,0.089132
2,0.226763,0.145437
1,0.095350,0.089132
11,0.095350,0.089132
4,0.095350,0.145437
7,0.095350,0.145437
3,0.096655,0.089132
6,0.096655,0.089132


In [174]:
X_test_encoded

,animal,locomocao
10,0.111111,0.145437
9,0.111111,0.145437
0,0.095350,0.089132
